# Projeto Risco (VaR e ES)
## Turma 780, Alunos:
- Alexandre Lima
- Mario Cesa
- Miguel Prytoluk
- Rogerio Penchel
### Objetivo: Construção de uma calculadora de retornos e VaR

### Bibliotecas utilizadas

In [1]:
import numpy as np
import pandas as pd

# import yfinance as yf
import investpy

### Baixa dados históricos do US$ dos últimos 5 anos
Utilizado nos testes

In [2]:
df_usd = investpy.get_currency_cross_historical_data(
        currency_cross=f'USD/BRL',
        from_date='01/01/2017',
        to_date='31/12/2029')

df_usd

,Open,High,Low,Close,Currency
Date,,,,,
2017-01-02,3.2538,3.2887,3.2516,3.2857,BRL
2017-01-03,3.2824,3.2929,3.2466,3.2652,BRL
2017-01-04,3.2517,3.2517,3.2100,3.2163,BRL
2017-01-05,3.2163,3.2294,3.1928,3.1991,BRL
2017-01-06,3.1972,3.2241,3.1871,3.2241,BRL
...,...,...,...,...,...
2022-02-23,5.0621,5.0621,4.9939,5.0097,BRL
2022-02-24,5.0050,5.1640,5.0050,5.1231,BRL
2022-02-25,5.1199,5.1810,5.0785,5.1623,BRL


### Função calcula_retorno

Função calcula_retorno(pandas serie_precos, holding_period)

Retorna: retorno simples (pd.Dataframe)

In [25]:
def calcula_retorno(serie_precos, holding_period):
    df = pd.DataFrame()
    df['return'] = serie_precos / serie_precos.shift(holding_period) - 1
    df['PnL'] = serie_precos - serie_precos.shift(holding_period)
    return df

In [26]:
# Testes da função calcula_retorno

df_ret = calcula_retorno(df_usd['Close'], 1)
df_ret

,return,PnL
Date,,
2017-01-02,NaN,NaN
2017-01-03,-0.006239,-0.0205
2017-01-04,-0.014976,-0.0489
2017-01-05,-0.005348,-0.0172
2017-01-06,0.007815,0.0250
...,...,...
2022-02-23,-0.009608,-0.0486
2022-02-24,0.022636,0.1134
2022-02-25,0.007652,0.0392


### Função calcula_log_retorno

Função calcula_log_retorno(pandas serie_precos, holding_period)

Retorna: log-retorno (pd.Dataframe)

In [30]:
def calcula_log_retorno(serie_precos, holding_period):
    df = calcula_retorno(serie_precos, holding_period)
    df['logret'] = np.log(serie_precos/serie_precos.shift(holding_period))
    return df

In [31]:
# Testes da função calcula_log_retorno

df_logret = calcula_log_retorno(df_usd['Close'], 1)
df_logret

,return,PnL,logret
Date,,,
2017-01-02,NaN,NaN,NaN
2017-01-03,-0.006239,-0.0205,-0.006259
2017-01-04,-0.014976,-0.0489,-0.015089
2017-01-05,-0.005348,-0.0172,-0.005362
2017-01-06,0.007815,0.0250,0.007784
...,...,...,...
2022-02-23,-0.009608,-0.0486,-0.009654
2022-02-24,0.022636,0.1134,0.022384
2022-02-25,0.007652,0.0392,0.007622


### Função calcula_var
Função: calcula_var(pandas serie_retornos, alpha, vlr_carteira_atual)

Retorna: VaR (float)


In [32]:
def calcula_var(serie_retornos, alpha, vlr_carteira_atual):
    return serie_retornos.quantile(alpha)

In [34]:
# Testes da função calcula_var

hp = 1
df_ret = calcula_retorno(df_usd['Close'], hp)

alpha = 0.05
dias = 1000
VaR = calcula_var(df_ret['PnL'].iloc[-dias:], alpha, 0)

print(f'VaR {(1-alpha)*100:0.0f}% || HP {hp} || {dias} dias: R$ {- VaR:.2f}')

VaR 95% || HP 1 || 1000 dias: R$ 0.09


### Função calcula_es
Função: calcula_es(pandas serie_retornos, alpha, vlr_carteira_atual)

Retorna VaR (float)

In [36]:
def calcula_es(serie_retornos, alpha, vlr_carteira_atual=1):
    amostra_es = serie_retornos[serie_retornos <= vlr_carteira_atual]
    return amostra_es.mean()*vlr_carteira_atual

In [37]:
# Testes da função calcula_es

hp = 1
df_ret = calcula_retorno(df_usd['Close'], hp)

alpha = 0.05
dias = 1000
es = calcula_es(df_ret['PnL'].iloc[-dias:], alpha, VaR)

print(f'ES {(1-alpha)*100:0.0f}%, HP {hp}, {dias} dias: R$ {- es:0.2f}')

ES 95%, HP 1, 1000 dias: R$ -0.01


In [35]:
df_ret['PnL'].iloc[-1000:]

Date
2018-04-25    0.0136
2018-04-26   -0.0095
2018-04-27   -0.0187
2018-04-30    0.0504
2018-05-01    0.0004
               ...  
2022-02-23   -0.0486
2022-02-24    0.1134
2022-02-25    0.0392
2022-02-28   -0.0024
2022-03-01    0.0006
Name: PnL, Length: 1000, dtype: float64